In [15]:
#pip install dask[complete] dask-ml pandas scikit-learn


In [25]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask_ml.model_selection as dml
from dask_ml.model_selection import train_test_split

import dask_ml.metrics as dmm
from dask_ml.linear_model import LogisticRegression
from dask.distributed import Client


In [17]:
# Cargar los datos con Dask
df = pd.read_csv('..\data\ENPOVE2022_V_200-300-400-500-600-700-800.csv')


In [35]:
# Definir las variables
variables_a_excluir = [
    'CCDD', 'DEPARTAMENTO', 'CCPP', 'PROVINCIA', 'CCDI', 'DISTRITO',
    'CIUDAD', 'CONGLOMERADO', 'NSELV', 'VIVIENDA', 'THOGAR', 'NHOGAR',
    'ESTRATO', 'VRESFIN', 'RESFIN', 'P15', 'P15_N', 'INF_200', 'P200_N',
    'P609_COD', 'P611_COD', 'P625A_COD_DEPA', 'P625_COD_PROV', 'P625_COD_DIST',
    'factorfinal', 'P602'
]

# Obtener la lista de todas las columnas
todas_las_columnas = df.columns.tolist()

# Determinar las covariables (X) excluyendo las especificadas
columnas_covariables = [col for col in todas_las_columnas if col not in variables_a_excluir]

# Definir predictores (X) y target (y)
X = df[columnas_covariables]
y = df['P602']  # Empleo

In [36]:

# Configuración del cliente Dask
client = Client()

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=4200, 
    shuffle=True  # Explicitly set shuffle
)
# Definir el modelo
model = LogisticRegression()

# Definir el grid para GridSearch
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10]
}

# Configurar GridSearchCV
grid_search = dml.GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Ajustar el modelo usando GridSearchCV
grid_search.fit(X_train, y_train)

# Selección del modelo óptimo
best_model = grid_search.best_estimator_

# Evaluación en el conjunto de entrenamiento
y_train_pred = best_model.predict(X_train)
train_accuracy = dmm.accuracy_score(y_train, y_train_pred)
train_roc_auc = dmm.roc_auc_score(y_train, best_model.predict_proba(X_train)[:, 1])

# Evaluación en el conjunto de prueba
y_test_pred = best_model.predict(X_test)
test_accuracy = dmm.accuracy_score(y_test, y_test_pred)
test_roc_auc = dmm.roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

# Imprimir resultados
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nIndicadores de calidad de ajuste en entrenamiento:")
print(f"Accuracy: {train_accuracy}")
print(f"ROC AUC: {train_roc_auc}")

print("\nIndicadores de calidad de ajuste en prueba:")
print(f"Accuracy: {test_accuracy}")
print(f"ROC AUC: {test_roc_auc}")

# Reporte de clasificación para la muestra de prueba
from dask_ml.metrics import classification_report
print("\nReporte de clasificación en prueba:")
print(classification_report(y_test, y_test_pred))


C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\deploy\spec.py:285: UserWarning: 
Could not launch service 'dashboard' on port 8787. Got the following message:

failed to validate _ServerOpts(...).prefix: expected a value of type str, got None of type NoneType
  self.scheduler = await self.scheduler
distributed.utils - ERROR - addresses should be strings or tuples, got None
Traceback (most recent call last):
  File "C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\utils.py", line 662, in log_errors
    yield
  File "C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\scheduler.py", line 2126, in remove_worker
    address = self.coerce_address(address)
  File "C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\scheduler.py", line 4836, in coerce_address
    raise TypeError("addresses should be strings or tuples, got %r" % (addr,))
TypeError: addresses should be strings or tuples, got None
distributed.core - ERROR - addresses should be strings or tuples, got None


distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\protocol\core.py", line 106, in loads
    header = msgpack.loads(header, use_list=False, **msgpack_opts)
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
ValueError: tuple is not allowed for map key when strict_map_key=True
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x0000020C4DD51FA0>>, <Task finished name='Task-3347' coro=<Nanny._on_exit() done, defined at C:\Users\Alvaro\anaconda3\lib\site-packages\distributed\nanny.py:388> exception=ValueError('tuple is not allowed for map key when strict_map_key=True')>)
Traceback (most recent call last):
  File "C:\Users\Alvaro\anaconda3\lib\site-packages\tornado\ioloop.py", line 741, in _run_callback
    ret = callback()
  File "C:\Users\Alvaro\anaconda

KeyboardInterrupt: 

## Limitaciones y posibles extensiones
1. El ajuste del modelo puede no ser óptimo si el grid de búsqueda es pequeño
2. Los indicadores de calidad de ajuste dependen de la calidad de los datos.
3. Se podría usar una mayor variedad de modelos y técnicas para comparación.
4. La escalabilidad puede ser un problema con datasets extremadamente grandes.